In [2]:
import glob
import json
import pandas as pd

months = sorted(glob.glob("./bbc-json/matches/*.json"))

match_ids = []

for month in months:
    with open(month) as f:
        data = json.load(f)
        if 'payload' in data.keys():
            tournaments = data['payload'][0]['body']['matchData']
        else:
            tournaments = data['matchData']
        for i in range(0, len(tournaments)):
            matches = tournaments[i]['tournamentDatesWithEvents'].keys()
            for match in matches:
                match_data = tournaments[i]['tournamentDatesWithEvents'][match]
                match_id = match_data[0]['events'][0]['eventKey']
                game_date = match_data[0]['events'][0]['startTime'].split('T')[0]
                match_ids.append((match_id, game_date))

match_ids_df = pd.DataFrame(match_ids, columns=['event_id', 'game_date'])

match_ids_df.head(3)

,event_id,game_date
0,EFBO361411,2011-08-06
1,EFBO361420,2011-08-13
2,EFBO361433,2011-08-16


In [16]:
line_ups = sorted(glob.glob("./bbc-json/lineups/*.json"))

all_apps_df = pd.DataFrame()
all_subs_df = pd.DataFrame()
all_sub_mins_df = pd.DataFrame()
all_yellows_df = pd.DataFrame()
all_reds_df = pd.DataFrame()

for line_up in line_ups:
    with open(line_up) as f:
        data = f.read()

    data = json.loads(data)
    try:
        event_id = data["event_id"]
    except:
        event_id = data[0]["event_id"]
        data = data[0]

    home_team = data["data"]["teams"]["homeTeam"]["name"]
    away_team = data["data"]["teams"]["awayTeam"]["name"]

    if home_team == "Tranmere Rovers":
        team = 'homeTeam'
    else:
        team = 'awayTeam'

    formation = data["data"]["teams"][team]["formation"]
    formation = "-".join(str(formation))

    players = data["data"]["teams"][team]["players"]

    apps_df = pd.DataFrame()
    cards_df = pd.DataFrame()
    subs_df = pd.DataFrame()
    sub_mins_df = pd.DataFrame()

    for player in players:
        player_name = player["name"]["full"]
        try:
            shirt_no = player["meta"]["uniformNumber"]
        except:
            shirt_no = None
        role = player["meta"]["status"].replace("bench", "sub")
        
        player_app = {
            "event_id": event_id,
            "player_name": player_name,
            "shirt_no": shirt_no,
            "role": role
        }
        player_app = pd.DataFrame(player_app, index = [0])

        apps_df = pd.concat([apps_df, player_app], axis = 0)

        cards = player["bookings"]
        if cards:
            player_cards = pd.json_normalize(player["bookings"])
            player_cards["event_id"] = event_id
            player_cards["player_name"] = player_name
            player_cards["card_min"] = player_cards["timeElapsed"]
            player_cards["card_inj_time"] = player_cards["addedTime"]
            cards_df = pd.concat([cards_df, player_cards], axis = 0)

        subs = player["substitutions"]
        if subs:
            sub_min = subs[0]["timeElapsed"]
            try:
                sub_on_no = subs[0]["replacedBy"]["meta"]["uniformNumber"]
            except:
                sub_on_no = None
            player_on = subs[0]["replacedBy"]["name"]["full"]
            sub_mins_pld = subs[0]["timeElapsed"]
            sub_mins_pld_inj = subs[0]["addedTime"]
            sub_mins = [{
                "event_id": event_id,
                "player_name": player_name,
                "min_off": sub_min,
                "min_on": "",
                "added_time": sub_mins_pld_inj
            },
            {
                "event_id": event_id,
                "player_name": player_on,
                "min_off": "",
                "min_on": sub_min,
                "added_time": sub_mins_pld_inj
            }]
            sub_mins = pd.DataFrame(sub_mins)
            sub_mins_df = pd.concat([sub_mins_df, sub_mins], axis = 0)

            sub_names = [{
                "event_id": event_id,
                "shirt_no": sub_on_no,
                "player_name": player_on,
                "on_for": shirt_no,
                "off_for": ""
            },
            {
                "event_id": event_id,
                "shirt_no": shirt_no,
                "player_name": player_name,
                "on_for": "",
                "off_for": sub_on_no
            }]
            sub_names = pd.DataFrame(sub_names)
            subs_df = pd.concat([subs_df, sub_names], axis = 0)

            sub_cards = subs[0]["replacedBy"]["bookings"]
            if sub_cards:
                sub_cards = pd.json_normalize(sub_cards)
                sub_cards["event_id"] = event_id
                sub_cards["player_name"] = player_on
                sub_cards["card_min"] = sub_cards["timeElapsed"]
                sub_cards["card_inj_time"] = sub_cards["addedTime"]
                cards_df = pd.concat([cards_df, sub_cards], axis = 0)

            # Second sub
            # Rare occasion where sub is subbed
            sub_subs = subs[0]["replacedBy"]["substitutions"]
            if sub_subs:
                sub_min = subs[1]["timeElapsed"]
                try:
                    sub_on_no = subs[1]["replacedBy"]["meta"]["uniformNumber"]
                except:
                    sub_on_no = None
                player_on2 = subs[1]["replacedBy"]["name"]["full"]
                sub_mins_pld = subs[1]["timeElapsed"]
                sub_mins_pld_inj = subs[1]["addedTime"]
                sub_mins = [{
                    "event_id": event_id,
                    "player_name": player_on,
                    "min_off": sub_min,
                    "min_on": "",
                    "added_time": sub_mins_pld_inj
                },
                {
                    "event_id": event_id,
                    "player_name": player_on2,
                    "min_off": "",
                    "min_on": sub_min,
                    "added_time": sub_mins_pld_inj
                }]
                sub_mins = pd.DataFrame(sub_mins)
                sub_mins_df = pd.concat([sub_mins_df, sub_mins], axis = 0)

                sub_names = [{
                    "event_id": event_id,
                    "shirt_no": sub_on_no,
                    "player_name": player_name,
                    "on_for": shirt_no,
                    "off_for": ""
                },
                {
                    "event_id": event_id,
                    "shirt_no": shirt_no,
                    "player_name": player_on,
                    "on_for": "",
                    "off_for": sub_on_no
                }]
                sub_names = pd.DataFrame(sub_names)
                subs_df = pd.concat([subs_df, sub_names], axis = 0)
    
    all_apps_df = pd.concat([all_apps_df, apps_df], axis = 0)
    all_subs_df = pd.concat([all_subs_df, subs_df], axis = 0)
    all_sub_mins_df = pd.concat([all_sub_mins_df, sub_mins_df], axis = 0)

    if not cards_df.empty:
        yellows_df = cards_df.query("type == 'yellow-card'")
        reds_df = cards_df.query("type.str.contains('red')")
        if not yellows_df.empty:
            yellows_df = yellows_df[["event_id", "player_name", "card_min", "card_inj_time"]]
            yellows_df["yellow_card"] = 1
            all_yellows_df = pd.concat([all_yellows_df, yellows_df], axis = 0)
            all_yellows_df = all_yellows_df[["event_id", "player_name", "yellow_card", "card_min", "card_inj_time"]]
        if not reds_df.empty:
            print(event_id)
            reds_df = reds_df[["event_id", "player_name", "timeElapsed", "card_inj_time"]].rename(columns = {"timeElapsed": "min_so"})
            all_reds_df = pd.concat([all_reds_df, reds_df], axis = 0)

# ALL REQUIRE GAME DATE
# all_subs_df
# all_sub_mins_df
# all_yellows_df
# all_reds_df

EFBO1064352
EFBO1064380
EFBO1064484
EFBO1064586
EFBO1064700
EFBO2040827
EFBO2130427
EFBO2130791
EFBO2213991
EFBO2214075
EFBO2214078
EFBO2214223
EFBO2214458
EFBO2301039
EFBO2301050
EFBO2301100
EFBO443878
EFBO443902
EFBO695861
EFBO695880
EFBO695940
EFBO695975
EFBO696027
EFBO696084
EFBO696120
EFBO696383
EFBO757039
EFBO757220
EFBO785977
EFBO814429
EFBO814517
EFBO864143
EFBO864170
EFBO864225
EFBO864249
EFBO864406
EFBO931203
EFBO931317
EFBO931466
EFBO959938
EFBO983127
EFBO992099
EFBO992195
EFBO992417


In [18]:
name_fixes = {
    'Owain Williams': 'Owain Fon Williams',
    'Jimmy McNulty': 'Jim McNulty',
    'Matthew Hill': 'Matt Hill',
    'Jack Flemming': 'Jack Fleming',
    'Jonathon Margetts': 'Johnny Margetts',
    'Jay McEveley': 'James McEveley',
    'Oliver Banks': 'Ollie Banks',
    'Corey Taylor': 'Corey Blackett-Taylor',
    'Nathaniel Knight-Percival': 'Nat Knight-Percival',
    'Joshua McPake': 'Josh McPake'
}

In [19]:
def fix_df(df):
    df = pd.merge(df, match_ids_df, on = 'event_id', how = 'left')
    df['player_name'] = df['player_name'].replace(name_fixes)
    return df.sort_values(by=['game_date']).drop_duplicates()

In [21]:
all_apps_df = fix_df(all_apps_df)
all_subs_df = fix_df(all_subs_df)
all_sub_mins_df = fix_df(all_sub_mins_df)
all_yellows_df = fix_df(all_yellows_df)
all_reds_df = fix_df(all_reds_df)

In [23]:
all_yellows_df.sort_values(by=['game_date', 'card_min', 'card_inj_time'])[['game_date', 'player_name', 'card_min', 'card_inj_time']].to_csv('basic-data/bbc-yellow-cards.csv', index = False)

In [24]:
all_reds_df.sort_values(by=['game_date', 'min_so', 'card_inj_time'])[['game_date', 'player_name', 'min_so', 'card_inj_time']].to_csv('basic-data/bbc-red-cards.csv', index = False)

In [25]:
all_apps_df.sort_values(by=['game_date', 'role', 'shirt_no'])[['game_date', 'player_name', 'shirt_no', 'role']].to_csv('basic-data/bbc-player-apps.csv', index = False)

In [26]:
all_subs_df[['game_date', 'shirt_no', 'player_name', 'on_for', 'off_for']].to_csv('basic-data/bbc-subs.csv', index = False)

In [27]:
all_sub_mins_df[['game_date', 'player_name', 'min_off', 'min_on', 'added_time']].to_csv('basic-data/bbc-sub-mins.csv', index = False)

In [29]:
match_ids_df.to_csv('basic-data/bbc-match-ids.csv', index = False)